In [ ]:
import numpy as np
import pandas as pd
import pybel
import torch
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from rdkit import Chem
from rdkit.Chem import AllChem


In [ ]:
df = pd.read_csv('data.csv')
df.head()


In [ ]:
def aminoacids(sequence):
    protein = ProteinAnalysis(sequence)
    aa_composition = protein.get_amino_acids_percent()
    feature_vector = np.array([aa_composition[aa]
                              for aa in 'ACDEFGHIKLMNPQRSTVWY'])
    mean = feature_vector.mean()
    std = feature_vector.std()
    feature_vector = (feature_vector - mean) / std
    return feature_vector


In [ ]:
data = df.values
data = np.array(data)
protiens = np.array([])
drugs = np.array([])
labels = np.array([])
for i in range(data.shape[0]):
    mol = Chem.MolFromSmiles(data[i, 0])
    adj_matrix = AllChem.GetAdjacencyMatrix(mol)
    protiens = np.append(protiens, adj_matrix)


for i in range(data.shape[0]):
    aminoacid = aminoacids(data[i, 1])
    drugs = np.append(drugs, aminoacid)

for i in range(data.shape[0]):
    label = data[i, 2]
    labels = np.append(labels, label)
